In [1]:
from IslandTime import retrieve_island_info, Segmentation, update_results_map, plot_shoreline_transects, Workflow, update_data_map, TimeSeriesCoastSat
import pandas as pd
import os
from tqdm import tqdm
import shapely
import geopandas as gpd
import matplotlib.pyplot as plt
from scipy import stats
import inspect
import numpy as np
from scipy.optimize import curve_fit
from scipy import ndimage
import osmnx as ox
import pyproj
%load_ext autoreload
%autoreload 2

In [9]:
gdf = gpd.read_file('shp//islands_seasonality_cond3_results.shp')

for i in range(len(gdf)):
    if gdf['geometry'][i] == shapely.geometry.Point(0, 0):
        # remove points
        gdf.drop(i, inplace=True)

In [3]:
gdf.plot(column='mBEAST', cmap='viridis', legend=True, markersize=1)

<Axes: >

In [9]:
def heatmap(d, bins=(100,100), smoothing=1.3, cmap='jet'):
    def getx(pt):
        return pt.coords[0][0]

    def gety(pt):
        return pt.coords[0][1]

    x = list(d.geometry.apply(getx))
    y = list(d.geometry.apply(gety))
    heatmap, xedges, yedges = np.histogram2d(y, x, bins=bins)
    extent = [yedges[0], yedges[-1], xedges[-1], xedges[0]]

    logheatmap = np.log(heatmap)
    logheatmap[np.isneginf(logheatmap)] = 0
    logheatmap = ndimage.filters.gaussian_filter(logheatmap, smoothing, mode='nearest')
    
    plt.imshow(logheatmap, cmap=cmap, extent=extent)
    plt.colorbar()
    plt.gca().invert_yaxis()
    plt.show()

In [37]:
heatmap(gdf[['mBEAST', 'geometry']], bins=50, smoothing=1.5)

In [39]:
# Count occurrences of each category
category_counts = gdf['mBEAST'].value_counts()

# Plot histogram
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
category_counts.plot(kind='bar', ax=ax, color='skyblue', edgecolor='black')


<Axes: xlabel='mBEAST'>

In [55]:
# Extract latitude from geometry
gdf['latitude'] = gdf.geometry.y
gdf['longitude'] = gdf.geometry.x

# Define latitude bins
bin_edges = np.linspace(gdf['latitude'].min(), gdf['latitude'].max(), num=9)  # Adjust the number of bins as needed
bin_edges_lon = np.linspace(gdf['longitude'].min(), gdf['longitude'].max(), num=9)  # Adjust the number of bins as needed
gdf['latitude_bin'] = pd.cut(gdf['latitude'], bins=bin_edges, include_lowest=True)
gdf['longitude_bin'] = pd.cut(gdf['longitude'], bins=bin_edges_lon, include_lowest=True)

# Group by latitude bin and category
grouped = gdf.groupby(['latitude_bin', 'mBEAST']).size().unstack(fill_value=0)
grouped_lon = gdf.groupby(['longitude_bin', 'mBEAST']).size().unstack(fill_value=0)

# Plot the grouped data
grouped.plot(kind='bar', stacked=True, figsize=(10, 6), colormap='Set1')
grouped_lon.plot(kind='bar', stacked=True, figsize=(10, 6), colormap='Set1')

# Customize the plot
plt.xlabel('Latitude Bin')
plt.ylabel('Count')
plt.title('Histogram of Categorical Values as a Function of Latitude')
plt.legend(title='Category')
plt.xticks(rotation=45, ha='right')
plt.show()

In [56]:
isl = 'Meragihuttaa'
gdf_isl = gdf[gdf['island'] == isl]

In [60]:
# reference shorelines dataset

rs = gpd.read_file('shp//islands_reference_shoreline.shp')

# select the island
rs_isl = rs[rs['island'] == isl]

In [68]:
# bounding box of reference shorelines

rs_isl.total_bounds

# create rectangle from bounding box
from shapely.geometry import box

rectangle = box(*rs_isl.total_bounds)

AttributeError: 'Polygon' object has no attribute 'plot'

In [4]:
huvadhoo_atoll = ox.geocode_to_gdf('Huvadhu Atoll, Maldives')
huvadhoo_atoll

,geometry,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,lat,lon,class,type,place_rank,importance,addresstype,name,display_name
0,"POLYGON ((72.98490 0.48726, 72.98744 0.48223, ...",0.921451,0.180742,73.581764,72.9849,366339373,way,671807065,0.552063,73.269225,natural,atoll,22,0.295576,archipelago,Huvadhu Atoll,"Huvadhu Atoll, North Huvadhu Atoll, Maldives"


In [79]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

rs_isl.plot(ax=ax, color='k', linewidth=1)
gdf_isl.plot(ax=ax, column='mBEAST', cmap='viridis', legend=True, markersize=30)
gpd.GeoSeries(rectangle).plot(ax=ax, color='none', edgecolor='r')
huvadhoo_atoll.plot(ax=ax, color='none', edgecolor='b')


<Axes: >

In [92]:
# Count occurrences of each category
category_counts = gdf_isl['mBEAST'].value_counts()

# Plot histogram
# 'line', 'bar', 'barh', 'kde', 'density', 'area', 'hist', 'box', 'pie', 'scatter', 'hexbin'
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
category_counts.plot(kind='bar', ax=ax, color='skyblue')

<Axes: xlabel='mBEAST'>

In [94]:
transects = island_info['spatial_reference']['transects']

In [97]:
import pyproj
# Define projection
src_crs = pyproj.CRS('EPSG:3857')
tgt_crs = pyproj.CRS('EPSG:4326')
transformer = pyproj.Transformer.from_crs(src_crs, tgt_crs, always_xy=True)

# Transform transects
transects_image_crs = {}
for key_transect, transect in transects.items():
    transect_x_transformed, transect_y_transformed = transformer.transform(transect[:, 0], transect[:, 1])
    transects_image_crs[key_transect] = np.column_stack((transect_x_transformed, transect_y_transformed))

# create dictionary of LineString
transects_linestring = {}
for key_transect, transect in transects_image_crs.items():
    transects_linestring[key_transect] = shapely.geometry.LineString(transect)

In [104]:
# find closest transect to points
gdf_isl['closest_transect'] = gdf_isl['geometry'].apply(lambda x: min(transects_linestring, key=lambda y: transects_linestring[y].distance(x)))

In [143]:
# calculate orientation of each transect
def calculate_orientation(transect):
    x1, y1 = transect.coords[0]
    x2, y2 = transect.coords[-1]
    angle = np.arctan2(y2 - y1, x2 - x1)

    # in positive degree, 0 to 360
    angle_deg = np.degrees(angle) % 360
    return angle_deg

# create dictionary of transect orientation
transects_orientation = {}
for key_transect, transect in transects_linestring.items():
    transects_orientation[key_transect] = calculate_orientation(transect)

In [141]:
transects_orientation[87]
gdf_isl.plot(column='orientation', cmap='viridis', legend=True, markersize=30)

<Axes: >

In [125]:
# group transect orientation in bins
bin_edges = np.linspace(0, 360, num=9)  # Adjust the number of bins as needed
gdf_isl['orientation_bin'] = pd.cut(gdf_isl['closest_transect'].map(transects_orientation), bins=bin_edges, include_lowest=True)

# Group by orientation bin and category
grouped = gdf_isl.groupby(['orientation_bin', 'mBEAST']).size().unstack(fill_value=0)

# Plot the grouped data
grouped.plot(kind='bar', stacked=True, figsize=(10, 6), colormap='Set1')

# plot in rose diagram
fig, ax = plt.subplots(1, 1, subplot_kw=dict(polar=True), figsize=(10, 6))

# Compute histogram
# color should be the category
n, bins, patches = ax.hist(np.radians(gdf_isl['closest_transect'].map(transects_orientation)),
                           bins=36, range=(0, 2 * np.pi), color='skyblue', edgecolor='black')

# Set the direction of the zero angle
ax.set_theta_zero_location('N')

# Set the clockwise direction
ax.set_theta_direction(-1)

# Customize the plot
plt.xlabel('Orientation')

Text(0.5, 0, 'Orientation')

In [144]:
gdf_isl['orientation'] = gdf_isl['closest_transect'].map(transects_orientation)

# Define orientation bins
num_bins = 36
bin_edges = np.linspace(0, 360, num_bins + 1)

# Bin the orientations
gdf_isl['orientation_bin'] = pd.cut(gdf_isl['orientation'], bins=bin_edges, include_lowest=True, labels=False)

# Group by orientation bin and category
grouped = gdf_isl.groupby(['orientation_bin', 'mBEAST']).size().unstack(fill_value=0)

# Convert bin numbers to radians
bin_centers = np.deg2rad(bin_edges[:-1] + np.diff(bin_edges) / 2)

In [145]:
# Define colors for each category
colors = plt.cm.Set1(np.linspace(0, 1, len(grouped.columns)))

fig, ax = plt.subplots(1, 1, subplot_kw=dict(polar=True), figsize=(10, 6))

# Plot each category as a stacked bar
bottom = np.zeros(num_bins)
for i, category in enumerate(grouped.columns):
    values = grouped[category].reindex(range(num_bins), fill_value=0).values
    bars = ax.bar(bin_centers, values, width=np.deg2rad(np.diff(bin_edges)), bottom=bottom, color=colors[i], edgecolor='black', label=category)
    bottom += values

# Set the direction of the zero angle
# ax.set_theta_zero_location('N')

# Set the clockwise direction
# ax.set_theta_direction(-1)

# Add legend
ax.legend(title='mBEAST', bbox_to_anchor=(1.1, 1.05))

# Customize the plot
plt.title('Stacked Rose Diagram of Transect Orientations by mBEAST Category')
plt.show()

In [10]:
src_crs = pyproj.CRS('EPSG:3857')
tgt_crs = pyproj.CRS('EPSG:4326')
transformer = pyproj.Transformer.from_crs(src_crs, tgt_crs, always_xy=True)

# calculate orientation of each transect
def calculate_orientation(transect):
    x1, y1 = transect.coords[0]
    x2, y2 = transect.coords[-1]
    angle = np.arctan2(y2 - y1, x2 - x1)

    # in positive degree, 0 to 360
    angle_deg = np.degrees(angle) % 360

    # nautical orientation
    angle_deg = (90 - angle_deg) % 360

    return angle_deg

def plot_rose_diagram(gdf, list_islands, reference_shorelines, lab_save, feature_res='mBEAST'):

    # Select island columns in gdf
    gdf_isl = gdf[gdf['island'].isin(list_islands)]

    # Select reference shorelines
    rs_isl = reference_shorelines[reference_shorelines['island'].isin(list_islands)]

    # Get transects
    for isl in list_islands:
        # Retrieve island information
        island_info = retrieve_island_info(isl, 'Maldives', verbose=False)
        transects = island_info['spatial_reference']['transects']

        # Subset gdf
        gdf_isl_subset = gdf_isl[gdf_isl['island'] == isl]

        # Transform transects
        transects_image_crs = {}
        for key_transect, transect in transects.items():
            transect_x_transformed, transect_y_transformed = transformer.transform(transect[:, 0], transect[:, 1])
            transects_image_crs[key_transect] = np.column_stack((transect_x_transformed, transect_y_transformed))
        
        # Create dictionary of LineString
        transects_linestring = {}
        for key_transect, transect in transects_image_crs.items():
            transects_linestring[key_transect] = shapely.geometry.LineString(transect)
        
        # Calculate orientation of each transect
        transects_orientation = {}
        for key_transect, transect in transects_linestring.items():
            transects_orientation[key_transect] = calculate_orientation(transect)
        
        # Find closest transect to points
        gdf_isl_subset['closest_transect'] = gdf_isl_subset['geometry'].apply(lambda x: min(transects_linestring, key=lambda y: transects_linestring[y].distance(x)))

        # Calculate orientation of each point
        gdf_isl_subset['orientation'] = gdf_isl_subset['closest_transect'].map(transects_orientation)

        # Define orientation bins
        num_bins = 36
        bin_edges = np.linspace(0, 360, num_bins + 1)
        
        # Bin the orientations
        gdf_isl_subset['orientation_bin'] = pd.cut(gdf_isl_subset['orientation'], bins=bin_edges, include_lowest=True, labels=False)

        # Merge gdf with next iteration
        if isl == list_islands[0]:
            gdf_isl_merged = gdf_isl_subset
        else:
            gdf_isl_merged = pd.concat([gdf_isl_merged, gdf_isl_subset])
    
    # Group by orientation bin and category
    grouped = gdf_isl_merged.groupby(['orientation_bin', feature_res]).size().unstack(fill_value=0)

    # Convert bin numbers to radians
    bin_centers = np.deg2rad(bin_edges[:-1] + np.diff(bin_edges) / 2)

    # Define colors for each category
    # colors = plt.cm.Set1(np.linspace(0, 1, len(grouped.columns)))

    # Create custom colormap
    dict_colours_labels = {'sw_monsoon': '#58508d',
                            'ne_monsoon': '#bc5090', 
                            'transition_sw_ne': '#ffa600',
                            'transition_ne_sw': '#ff6361',
                            'undetermined': '#003f5c'}

    dict_labels = {'sw_monsoon': 'SW Monsoon',
                     'ne_monsoon': 'NE Monsoon',
                     'transition_sw_ne': 'SW-NE Transition',
                     'transition_ne_sw': 'NE-SW Transition',
                     'undetermined': 'Undetermined'}

    colors = [dict_colours_labels[col] for col in grouped.columns]

    fig, ax = plt.subplots(1, 1, subplot_kw=dict(polar=True), figsize=(10, 6))

    # Plot each category as a stacked bar
    bottom = np.zeros(num_bins)

    for i, category in enumerate(grouped.columns):
        values = grouped[category].reindex(range(num_bins), fill_value=0).values
        bars = ax.bar(bin_centers, values, width=np.deg2rad(np.diff(bin_edges)), bottom=bottom, color=colors[i], edgecolor='black', label=dict_labels[category], zorder=3)
        bottom += values
    ax.grid(zorder=0)
    
    # increase tick size & bold
    ax.tick_params(axis='both', which='major', labelsize=15, colors='white')

    # tick in white


    # ax.legend(fontsize=15)

    ax.set_theta_zero_location('N')
    ax.set_theta_direction(-1)

    fig.savefig('figures//atoll//rose_diagram_cond3_{}_{}_N.png'.format(feature_res, lab_save), dpi=300, bbox_inches='tight', transparent=True)

# reference shorelines dataset
rs = gpd.read_file('shp//islands_reference_shoreline.shp')

# plot_rose_diagram(gdf=gdf, list_islands=['Meragihuttaa'], reference_shorelines=rs, lab_save='tt', feature_res='mBEAST')
# gdf, list_islands, reference_shorelines, lab_save, feature_res='mBEAST'

In [11]:
import numpy as np
import pandas as pd
import shapely.geometry
import matplotlib.pyplot as plt
import pyproj

src_crs = pyproj.CRS('EPSG:3857')
tgt_crs = pyproj.CRS('EPSG:4326')
transformer = pyproj.Transformer.from_crs(src_crs, tgt_crs, always_xy=True)

# calculate orientation of each transect
def calculate_orientation(transect):
    x1, y1 = transect.coords[0]
    x2, y2 = transect.coords[-1]
    angle = np.arctan2(y2 - y1, x2 - x1)

    # in positive degree, 0 to 360
    angle_deg = np.degrees(angle) % 360

    # nautical orientation
    angle_deg = (90 - angle_deg) % 360

    return angle_deg

def plot_classified_amplitude_rose_diagram(gdf, list_islands, reference_shorelines, lab_save, amplitude_column='aBEAST'):

    # Define amplitude categories
    # amplitude_bins = [0, 7.5, 15, 22.5, 30, np.inf]
    amplitude_bins = [0, 5, 10, 15, 20, 25, 30, 35, 40, np.inf]
    amplitude_labels = ['<5', '5-10', '10-15', '15-20', '20-25', '25-30', '30-35', '35-40', '>40']

    # Select island columns in gdf
    gdf_isl = gdf[gdf['island'].isin(list_islands)]

    # Select reference shorelines
    rs_isl = reference_shorelines[reference_shorelines['island'].isin(list_islands)]

    # Get transects
    for isl in list_islands:
        # Retrieve island information
        island_info = retrieve_island_info(isl, 'Maldives', verbose=False)
        transects = island_info['spatial_reference']['transects']

        # Subset gdf
        gdf_isl_subset = gdf_isl[gdf_isl['island'] == isl]

        # Transform transects
        transects_image_crs = {}
        for key_transect, transect in transects.items():
            transect_x_transformed, transect_y_transformed = transformer.transform(transect[:, 0], transect[:, 1])
            transects_image_crs[key_transect] = np.column_stack((transect_x_transformed, transect_y_transformed))
        
        # Create dictionary of LineString
        transects_linestring = {}
        for key_transect, transect in transects_image_crs.items():
            transects_linestring[key_transect] = shapely.geometry.LineString(transect)
        
        # Calculate orientation of each transect
        transects_orientation = {}
        for key_transect, transect in transects_linestring.items():
            transects_orientation[key_transect] = calculate_orientation(transect)
        
        # Find closest transect to points
        gdf_isl_subset['closest_transect'] = gdf_isl_subset['geometry'].apply(lambda x: min(transects_linestring, key=lambda y: transects_linestring[y].distance(x)))

        # Calculate orientation of each point
        gdf_isl_subset['orientation'] = gdf_isl_subset['closest_transect'].map(transects_orientation)

        # Classify amplitudes
        gdf_isl_subset['amplitude_category'] = pd.cut(gdf_isl_subset[amplitude_column], bins=amplitude_bins, labels=amplitude_labels)

        # Merge gdf with next iteration
        if isl == list_islands[0]:
            gdf_isl_merged = gdf_isl_subset
        else:
            gdf_isl_merged = pd.concat([gdf_isl_merged, gdf_isl_subset])
    
    # Define orientation bins
    num_bins = 36
    bin_edges = np.linspace(0, 360, num_bins + 1)
    
    # Bin the orientations
    gdf_isl_merged['orientation_bin'] = pd.cut(gdf_isl_merged['orientation'], bins=bin_edges, include_lowest=True, labels=False)

    # Group by orientation bin and amplitude category
    grouped = gdf_isl_merged.groupby(['orientation_bin', 'amplitude_category']).size().unstack(fill_value=0)

    # Convert bin numbers to radians
    bin_centers = np.deg2rad(bin_edges[:-1] + np.diff(bin_edges) / 2)

    # Define colors for each category
    colors = plt.cm.plasma(np.linspace(0, 1, len(grouped.columns)))

    fig, ax = plt.subplots(1, 1, subplot_kw=dict(polar=True), figsize=(10, 6))

    # Plot each amplitude category as a stacked bar
    bottom = np.zeros(num_bins)
    for i, category in enumerate(grouped.columns):
        values = grouped[category].reindex(range(num_bins), fill_value=0).values
        bars = ax.bar(bin_centers, values, width=np.deg2rad(np.diff(bin_edges)), bottom=bottom, color=colors[i], edgecolor='black', label=category, zorder=3)
        bottom += values

    ax.grid(zorder=0)
    ax.tick_params(axis='both', which='major', labelsize=15, colors='white')
    ax.set_theta_zero_location('N')
    ax.set_theta_direction(-1)
# 
    # ax.legend(fontsize=15)
    fig.savefig('figures//atoll//rose_diagram_cond3_aBEAST_{}_N.png'.format(lab_save), dpi=300, bbox_inches='tight', transparent=True)

# reg = 'west_atoll'

# # read north atoll
# # gdf_north_atoll = gpd.read_file('shp//atoll//north_atoll.shp')
# gdf_west_atoll = gpd.read_file('shp//atoll//{}.shp'.format(reg))

# # from gdf, list all islands inside the north atoll
# # isl_inside_north_atoll = gdf_north_atoll.geometry[0].contains(gdf.geometry)
# isl_inside_west_atoll = gdf_west_atoll.geometry[0].contains(gdf.geometry)

# # List all islands inside the north atoll
# # list_islands_north_atoll = gdf[isl_inside_north_atoll]['island'].unique()
# list_islands_west_atoll = gdf[isl_inside_west_atoll]['island'].unique()

# plot_classified_amplitude_rose_diagram(gdf, list_islands_west_atoll.tolist(), reference_shorelines=rs, lab_save=reg)

In [13]:
for reg in ['north_atoll', 'south_atoll', 'west_atoll', 'east_atoll', 'inside_atoll']:
# for reg in ['north_atoll']:
    # read north atoll
    # gdf_north_atoll = gpd.read_file('shp//atoll//north_atoll.shp')
    gdf_west_atoll = gpd.read_file('shp//atoll//{}.shp'.format(reg))

    # from gdf, list all islands inside the north atoll
    # isl_inside_north_atoll = gdf_north_atoll.geometry[0].contains(gdf.geometry)
    isl_inside_west_atoll = gdf_west_atoll.geometry[0].contains(gdf.geometry)

    # List all islands inside the north atoll
    # list_islands_north_atoll = gdf[isl_inside_north_atoll]['island'].unique()
    list_islands_west_atoll = gdf[isl_inside_west_atoll]['island'].unique()

    plot_rose_diagram(gdf=gdf, list_islands=list_islands_west_atoll.tolist(), reference_shorelines=rs, lab_save=reg, feature_res='pBEAST')

In [14]:
# for reg in ['north_atoll', 'south_atoll', 'inside_north_middle_atoll', 'inside_south_atoll', 'west_atoll', 'east_atoll']:
# for reg in ['north_atoll', 'south_atoll', 'west_atoll', 'east_atoll', 'inside_atoll']:
rs = gpd.read_file('shp//islands_reference_shoreline.shp')
for reg in ['north_atoll', 'south_atoll', 'west_atoll', 'east_atoll', 'inside_atoll']:

    # read north atoll
    # gdf_north_atoll = gpd.read_file('shp//atoll//north_atoll.shp')
    gdf_west_atoll = gpd.read_file('shp//atoll//{}.shp'.format(reg))

    # from gdf, list all islands inside the north atoll
    # isl_inside_north_atoll = gdf_north_atoll.geometry[0].contains(gdf.geometry)
    isl_inside_west_atoll = gdf_west_atoll.geometry[0].contains(gdf.geometry)

    # List all islands inside the north atoll
    # list_islands_north_atoll = gdf[isl_inside_north_atoll]['island'].unique()
    list_islands_west_atoll = gdf[isl_inside_west_atoll]['island'].unique()

    plot_classified_amplitude_rose_diagram(gdf, list_islands_west_atoll.tolist(), reference_shorelines=rs, lab_save=reg)

In [13]:
# create a legend and save it as a figure

fig, ax = plt.subplots(1, 1, figsize=(10, 6))

# Create custom colormap
# Create custom colormap
dict_colours_labels = {'sw_monsoon': '#58508d',
                        'ne_monsoon': '#bc5090', 
                        'transition_sw_ne': '#ffa600',
                        'transition_ne_sw': '#ff6361',
                        'undetermined': '#003f5c'}

dict_labels = {'sw_monsoon': 'SW Monsoon',
                    'ne_monsoon': 'NE Monsoon',
                    'transition_sw_ne': 'SW-NE Transition',
                    'transition_ne_sw': 'NE-SW Transition',
                    'undetermined': 'Undetermined'}

# create a legend and save it as a figure

for i, (key, value) in enumerate(dict_labels.items()):
    ax.bar(0, 0, color=dict_colours_labels[key], label=value)

ax.set_axis_off()
ax.legend(loc='center', fontsize=15)
fig.savefig('figures//atoll//legend_rose_diagram.png', dpi=300, bbox_inches='tight', transparent=True)







In [15]:
def save_legend_figure(amplitude_labels, colors, filename):
    fig, ax = plt.subplots(1, 1, figsize=(10, 6))
    for i, label in enumerate(amplitude_labels):
        ax.bar(0, 0, color=colors[i], label=label)
    ax.set_axis_off()
    ax.legend(loc='center', fontsize=15)
    fig.savefig(filename, dpi=300, bbox_inches='tight', transparent=True)

# Example usage for plot:
# plot_classified_amplitude_rose_diagram(gdf, list_islands, reference_shorelines, 'example_save', 'amplitude')

# Example usage for legend:
amplitude_bins = [0, 5, 10, 15, 20, 25, 30, 35, 40, np.inf]
amplitude_labels = ['<5', '5-10', '10-15', '15-20', '20-25', '25-30', '30-35', '35-40', '>40']
cmap = plt.get_cmap('plasma')
colors = cmap(np.linspace(0, 1, len(amplitude_labels)))
save_legend_figure(amplitude_labels, colors, 'figures//atoll//legend_rose_diagram_amplitude.png')